In [1]:

import gc
import json
import math
import os
import pickle
import random
import re
import sys

import numpy as np
from charset_normalizer import from_path
from gensim.corpora import Dictionary
import nltk
from tqdm import tqdm

"""
Se importan las librerias que se necesiten, 
si se quiere ejecutar el notebook, se recomienda crear la carpeta de data, y poner ahi los files como se describe

"""
ACTUAL_PATH = os.getcwd()
# Donde esta el 20 News
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
# Donde se encuentra el BAC
PATH_BAC = os.path.join(ACTUAL_PATH, "data/BAC/blogs")
# Donde se van a guardar los files que se van obteniendo
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH, "data/final_files")


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')




### Upload 20N 

In [2]:
"""Para esto se define en que formato y donde se quiere el archivo completo de 20N"""

NEW_20N_FILE = os.path.join(PATH_FINAL_FILES, "20N.jsonl")

mayor_folders_20N = os.listdir(PATH_20N)
dictionary = {}
"""
Para cada archivo disponible en 20N,  
se generan registros con:  
- el ID del archivo  
- el tema del archivo  
- el texto del contenido  

Todo se guarda en formato JSONL por cuestiones de formato.
"""
with open(NEW_20N_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for folder in mayor_folders_20N:
        minor_files_path = os.path.join(PATH_20N, folder)
        minor_files = os.listdir(minor_files_path)
        for file in minor_files:
            file_path = os.path.join(minor_files_path, file)
            with open(file_path, "r", encoding="utf-8", errors="replace") as f:
                text = f.read().strip()

            record = {"id": file, "theme": folder, "text": text}
            unit = folder + file
            if file in dictionary.keys():
                dictionary[unit] += 1
            else:
                dictionary[unit] = 1
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")

In [ ]:
def preprocess_text(text: str) -> list[str]:
    """procesador de sentencias

    Args:
        text (str): sentencia a procesar

    Returns:
        list[str]: lista de palabras y simbolos a dejar.
    """
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s.,!?]", " ", text, flags=re.I | re.A | re.MULTILINE)
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"\d+", "NUM", text)
    text = "<s> " + text + " </s>"
    text = text.strip().split()
    return text


def divide_sentences(text: str) -> list[str]:
    """Divide los textos en sentencias

    Args:
        text (str): texto completo

    Returns:
        list[str]: Lista completa de sentencias
    """
    sentences = nltk.word_tokenize(text)
    return sentences

In [6]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")

def preprocess_text(text: str) -> list[str]:
    """
    """
    text = text.lower()
    text = re.sub(r"[^a-z0-9'\-]", " ", text)
    text = re.sub(r"\d+", "NUM", text) 
    text = re.sub(r"\s+", " ", text).strip() 

    tokens = nltk.word_tokenize(text, language="english")
    
    tokens = [stemmer.stem(token) for token in tokens if token.isalpha()]
    
    return tokens

In [8]:
with open(os.path.join(PATH_FINAL_FILES, "20N.jsonl")) as f:
    print(preprocess_text(f.readline()))
    print(f.readline())

['id', 'num', 'theme', 'misc', 'forsal', 'text', 'from', 'xorcist', 'cyberden', 'sf', 'ca', 'us', 'nsubject', 'for', 'sale', 'em', 'freq', 'shifter', 'n', 'nthis', 'is', 'yet', 'for', 'a', 'friend', 'again', 'n', 'nem', 'freq', 'shifter', 'the', 'machin', 'that', 'made', 'those', 'swirl', 'guitar', 'effect', 'n', 'way', 'back', 'in', 'the', 'num', 'n', 'nspacial', 'panner', 'with', 'harmon', 'shift', 'veri', 'rare', 'a', 'collector', 'item', 'this', 'nis', 'the', 'last', 'unit', 'em', 'ever', 'made', 'rack', 'mountabl', 'mono', 'in', 'up', 'down', 'signal', 'out', 'nwith', 'seper', 'pan', 'out', 'and', 'sine', 'unsin', 'voltag', 'out', 'vco', 'input', 'too', 'to', 'control', 'nlfo', 'if', 'interest', 'contact', 'kevin', 'befor', 'num', 'pm', 'pst', 'california', 'at', 'nand', 'make', 'an', 'offer', 'n', 'ndo', 'not', 'repli', 'to', 'this', 'account', 'n', 'nhave', 'a', 'nice', 'day', 'n', 'n', 'n', 'n', 'h', 'e', 'y', 'b', 'e', 'r', 'e', 'n', 'xorcist', 'cyberden', 'sf', 'ca', 'us', 'n

## I. For the 20N dataset compare two classifiers NB and LR to identify the 20 different newsgroups.

### Create your own processing pipeline for the task and justify it

### Divide the dataset into training (60%), validation (10%) and test (30%).

### Train NB and LR using the following vector representations:

#### tf (counts) representation (sklearn: CountVectorizer).

#### tfidf representation (sklearn: TfidfVectorizer).